# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
import joblib, pickle

ModuleNotFoundError: No module named 'azureml'

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [9]:
subscription_id = 'b4a122b5-b4d5-40e7-9878-57b87adf4a8b'
resource_group = 'aml-quickstarts-125265'
workspace_name = 'quick-starts-ws-125265'

ws = Workspace.get(name="quick-starts-ws-125265")

ds = Dataset.get_by_name(ws, name='mpg')
ds = ds.to_pandas_dataframe()

ds.columns=['mpg','cylinders','displacement','horsepower','weight','acceleration','model year','origin','car name']
ds = ds.replace('?',0)
ds.dropna(inplace=True)

ds.drop(columns=['car name'],inplace=True)
ds['mpg'] = x_df['mpg'].astype(float)


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=20,
    task='regression',
    primary_metric='normalized_mean_absolute_error',
    training_data=ds,
    label_column_name='mpg',
    n_cross_validations=3)

In [8]:
# TODO: Submit your experiment
autoexp = Experiment(workspace=ws,name='Ranga')
remote_run = autoexp.submit(automl_config)
remote_run.wait_for_completion(show_output=True)

ValidationException: ValidationException:
	Message: Install the required versions of packages using the requirements file. Requirements file location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, use remote target to avoid dependency management. 
Required version/Installed version
cryptography<=3.1.1/cryptography 3.2
zipp<=3.3.1/zipp 3.4.0
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Install the required versions of packages using the requirements file. Requirements file location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, use remote target to avoid dependency management. \nRequired version/Installed version\ncryptography<=3.1.1/cryptography 3.2\nzipp<=3.3.1/zipp 3.4.0",
        "inner_error": {
            "code": "NotSupported",
            "inner_error": {
         

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best,fitted = remote_run.get_output()
print(best)

In [ ]:
print(fitted)

In [ ]:
#TODO: Save the best model
filename = 'finalized_model.sav'
pickle.dump(best, open(filename, 'wb'))

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Register the model
registered_model = remote_run.register_model(model_name='finalized_model.sav',
                           tags={'area': 'mileage'},
                           model_path='best_automl.pkl')
print(registered_model.name, registered_model.id, registered_model.version, sep='\t')

#Create inference config
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.get(ws, "RangaENV")

inference_config = InferenceConfig(entry_script='train_FINAL.py',
                                    environment=env)

#Deploy model and check service state
from azureml.core.webservice import LocalWebservice, Webservice

deployment_config = LocalWebservice.deploy_configuration(port=9030) #No authentication.
service = Model.deploy(ws, "deployed", [registered_model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests
import json

# URL for the web service
scoring_uri = '<your web service URI>'

# Two sets of data to score, so we get two results back
data = {"data":
            [
                18,
                4,
                122,
                87,
                2332,
                12,
                70,
                2,
                'chevrolet chevelle malibu'
            ]
        
        }
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

In [ ]:
service.delete()